In [1]:
import os
import boto3
import numpy as np
import tensorflow as tf
import sagemaker
from sagemaker import get_execution_role

sagemaker_session = sagemaker.Session()

role = get_execution_role()
region = sagemaker_session.boto_session.region_name


In [12]:
training_data_uri = 's3://sagemaker-us-east-1-731833107751/mnist-data'
print(training_data_uri)

s3://sagemaker-us-east-1-731833107751/mnist-data


In [2]:
# Store in S3

# s3 = boto3.resource("s3")
# for bucket in s3.buckets.all():
#     print(bucket.name)
    

amplify-my-app-dev-204117-deployment
amplify-rnamplify-dev-150850-deployment
amplify-rnamplifyweek-testing-111251-deployment
astoria-event-dev-serverlessdeploymentbucket-1txdvzi3u1fo6
elasticbeanstalk-us-east-1-731833107751
renegmed-aicamp
renegmed-sagemaker-container-webinar
sagemaker-us-east-1-731833107751


In [17]:
fashion_mnist = tf.keras.datasets.fashion_mnist
(x_train, y_train), (x_test, y_test) = fashion_mnist.load_data()

np.save('/tmp/x_train', x_train)
np.save('/tmp/y_train', y_train)
np.save('/tmp/x_test', x_test)
np.save('/tmp/y_test', y_test)
#s3.Bucket('sagemaker-us-east-1-731833107751').upload_file('ec2_file', 's3_file')

In [18]:
s3.Bucket('sagemaker-us-east-1-731833107751').upload_file('/tmp/x_train.npy', 'mnist-data/x_train.npy') 
s3.Bucket('sagemaker-us-east-1-731833107751').upload_file('/tmp/y_train.npy', 'mnist-data/y_train.npy')
s3.Bucket('sagemaker-us-east-1-731833107751').upload_file('/tmp/x_test.npy', 'mnist-data/x_test.npy')
s3.Bucket('sagemaker-us-east-1-731833107751').upload_file('/tmp/y_test.npy', 'mnist-data/y_test.npy')

In [14]:
# TensorFlow 2.0 script
!pygmentize 'mnist-2.py'

import tensorflow as tf
import argparse
import os
import numpy as np
import json

#import matplotlib.pyplot as plt 
#import subprocess

from tensorflow.keras.layers import Input, Conv2D, Dense, Flatten, Dropout
from tensorflow.keras.models import Model


def model(x_train, y_train, x_test, y_test):
    """Generate a simple model"""
#     model = tf.keras.models.Sequential([
#         tf.keras.layers.Flatten(),
#         tf.keras.layers.Dense(1024, activation=tf.nn.relu),
#         tf.keras.layers.Dropout(0.4),
#         tf.keras.layers.Dense(10, activation=tf.nn.softmax)
#     ])

#     model.compile(optimizer='adam',
#                   loss='sparse_categorical_crossentropy',
#                   metrics=['accuracy'])
#     model.fit(x_train, y_train)
#     model.evaluate(x_test, y_test)

#     return model

    K = len(set(y_train))
    # Build the model using the functional API
    i = Input(shape=x_train[0].shape)  # tensorflow.keras.layers.Input
    x = Conv2D(32, (3, 3), strides=2

In [22]:
from sagemaker.tensorflow import TensorFlow
mnist_estimator2 = TensorFlow(entry_point='mnist-2.py',
                             role=role,
                             train_instance_count=1,
                             train_instance_type='local',
                             #train_instance_type='ml.m5.4xlarge',
                             framework_version='2.0.0',
                             py_version='py3',
                             distributions={'parameter_server': {'enabled': True}})

In [ ]:
mnist_estimator2.fit(training_data_uri)

Creating tmp53_xy1tq_algo-1-pqh24_1 ... 
Attaching to tmp53_xy1tq_algo-1-pqh24_12mdone
algo-1-pqh24_1  | 2020-01-31 21:32:09,426 sagemaker-containers INFO     Imported framework sagemaker_tensorflow_container.training
algo-1-pqh24_1  | 2020-01-31 21:32:09,434 sagemaker-containers INFO     No GPUs detected (normal if no gpus installed)
algo-1-pqh24_1  | 2020-01-31 21:32:09,652 sagemaker-containers INFO     No GPUs detected (normal if no gpus installed)
algo-1-pqh24_1  | 2020-01-31 21:32:09,674 sagemaker-containers INFO     No GPUs detected (normal if no gpus installed)
algo-1-pqh24_1  | 2020-01-31 21:32:09,693 sagemaker-containers INFO     No GPUs detected (normal if no gpus installed)
algo-1-pqh24_1  | 2020-01-31 21:32:09,705 sagemaker-containers INFO     Invoking user script
algo-1-pqh24_1  | 
algo-1-pqh24_1  | Training Env:
algo-1-pqh24_1  | 
algo-1-pqh24_1  | {
algo-1-pqh24_1  |     "additional_framework_parameters": {
algo-1-pqh24_1  |         "sagemaker_parameter_server_enabled": 

In [ ]:
predictor2 = mnist_estimator2.deploy(initial_instance_count=1, instance_type='ml.c5.xlarge')

In [ ]:
# # Save the model to a temporary directory
# import tempfile

# MODEL_DIR = tempfile.gettempdir()
# version = 1
# export_path = os.path.join(MODEL_DIR, str(version))

# print('export_path = {}\n'.format(export_path))
# if os.path.isdir(export_path):
#     print('\nAlready saved a model, cleaning up\n')
#     !rm -r {export_path}
    
# tf.saved_model.save(model, export_path)
# # model.save(export_path)

# print('\nSaved model:')
# !ls -l {export_path}

In [ ]:
#tf.saved_model.contains_saved_model(export_path)


In [ ]:
#!saved_model_cli show --dir {export_path} --all


In [ ]:
#new_model = tf.saved_model.load(export_path)

In [ ]:
#outputs = new_model(x_test[1])

In [ ]:
#f = new_model.signatures["serving_default"]
#print(f(x=tf.constant([[1.]])))